<a href="https://colab.research.google.com/github/Tessa18/Works/blob/main/gpt2_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers --upgrade

In [2]:
!pip install -U PyPDF2
!pip install python-docx

In [3]:
import pandas as pd
import numpy as np
import re
from PyPDF2 import PdfReader
import os
import docx

In [4]:
def read_pdf(file_path):
  with open(file_path, "rb") as file:
    pdf_reader = PdfReader(file)
    text = ""
    for page_num in range(len(pdf_reader.pages)):
      text +=pdf_reader.pages[pages_num].extract_text()
  return text

def read_word(file_path):
  doc =docx.Document(file_path)
  text = ""
  for paragraph in doc.paragraphs:
    text +=paragraph.text +"\n"
  return text

def read_text(file_path):
  with open(file_path, "r") as file:
    text = file.read()
  return text

def read_documents_from_directory(directory):
  combined_text = ""
  for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    if filename.endswith(".pdf"):
        combined_text += read_pdf(file_path)
    elif filename.endswith(".docx"):
        combined_text +=read_word(file_path)
    elif filename.endswith(".txt"):
        combined_text +=  read_text(file_path)
  return combined_text


In [5]:
train_directory = '/content/drive/MyDrive/Colab Notebooks/training'
text_data = read_documents_from_directory(train_directory)
text_data = re.sub(r'\n+', '\n',text_data).strip()

In [6]:
with open("/content/drive/MyDrive/Colab Notebooks/training/train1.txt", "w") as f:
  f.write(text_data)

In [7]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [8]:
!pip install accelerate==0.27.2

In [9]:
def load_dataset(file_path, tokenizer, block_size = 128):
  dataset = TextDataset(
      tokenizer = tokenizer,
      file_path = file_path,
      block_size=block_size,
  )
  return dataset

In [10]:
def load_data_collator(tokenizer, mlm = False):
  data_collator = DataCollatorForLanguageModeling(
      tokenizer=tokenizer,
      mlm=mlm,
  )
  return data_collator

In [11]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
      output_dir=output_dir,
      overwrite_output_dir=overwrite_output_dir,
      per_device_train_batch_size=per_device_train_batch_size,
      num_train_epochs=num_train_epochs,
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      data_collator=data_collator,
      train_dataset=train_dataset,
  )
  trainer.train()
  trainer.save_model()

In [14]:
train_file_path = "/content/drive/MyDrive/Colab Notebooks/training/train1.txt"
model_name = 'gpt2'
output_dir = '/content/drive/MyDrive/Colab Notebooks/output'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 50.0
save_steps = 50000

In [15]:
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

Step,Training Loss


In [16]:
from transformers  import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [57]:
def load_model(model_path):
  model = GPT2LMHeadModel.from_pretrained(model_path)
  return model

def load_tokenizer(tokenizer_path):
  tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
  return tokenizer

def generate_text(model_path, sequence, max_length):
  model = load_model(model_path)
  tokenizer = load_tokenizer(model_path)
  ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
  final_outputs = model.generate(
      ids,
      do_sample=True,
      max_length=max_length,
      pad_token_id=model.config.eos_token_id,
      top_k=50,
      top_p=0.95
  )
  print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

In [64]:
model2_path = "/content/drive/MyDrive/Colab Notebooks/output"
sequence2 = {"QueryText": "how to control fruit borer in brinjal"}
max_len = 70
generate_text(model2_path, sequence2, max_len)

{'QueryText': 'how to control fruit borer in brinjal'}
 },
 {
  "QueryText": "how to control fruit borer in brinjal",
  "KccAns": "apply to you spray monocrotophos 30 mlpump"
 },
 {

